In [ ]:
import os
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import mne

from mne import find_events, Epochs, compute_covariance, make_ad_hoc_cov
from mne.datasets import sample
from mne.simulation import (simulate_sparse_stc, simulate_raw,
                            add_noise, add_ecg, add_eog)
import numpy as np
import matplotlib.pyplot as plt

from config2 import*

# raw_fname = pjoin(path_data, subjects[0], 'raw_VisuoMotor_no_hpass.fif')
# raw = mne.io.read_raw_fif(raw_fname)
from mne.datasets import sample

data_path = sample.data_path()
meg_path = data_path / 'MEG' / 'sample'
raw_fname = meg_path / 'sample_audvis_raw.fif'
fwd_fname = meg_path / 'sample_audvis-meg-eeg-oct-6-fwd.fif'

# Load real data as the template
raw = mne.io.read_raw_fif(raw_fname)

raw.crop(0,5)
raw.load_data()
raw = raw.resample(256)


fname_info = meg_path / 'sample_audvis_raw.fif'
info = mne.io.read_info(fname_info)
tstep = 1 / info['sfreq']


info = raw.info
info
#raw.info.ch_names#['UPPT001']



In [ ]:
# TODO: create src
# organize into epochs and add events
# do I have to organize df as well?

### using our subj

In [ ]:
subjects_dir = '/home/demitau/data_Quentin/subjects'
source_space_noCB = mne.setup_source_space(subject, spacing='oct4', add_dist=True,
                                 subjects_dir=subjects_dir, n_jobs = -1)
src = source_space_noCB

### Using sample subj

In [ ]:
data_path = sample.data_path()
subjects_dir = data_path / 'subjects'
subject = 'sample'
meg_path = data_path / 'MEG' / subject



#fname_info = meg_path / 'sample_audvis_raw.fif'
#info = mne.io.read_info(fname_info)

fwd_fname = meg_path / 'sample_audvis-meg-eeg-oct-6-fwd.fif'
fwd = mne.read_forward_solution(fwd_fname)
src = fwd['src']

#help(mne.create_info)

### try stimfilter

In [ ]:
raw_stimch.filter(picks = None, l_freq=raw.info['lowpass'], h_freq=None)

In [ ]:
chd_stim = np.zeros( (1,len(raw.times)), dtype=int)
chn_stim = 'SMTH'
info = mne.create_info([chn_stim], raw.info['sfreq'], ['stim'])
infopars = ['dev_head_t', 'lowpass']
for ip in infopars:
    info[ip] = raw.info[ip]
raw_stimch = mne.io.RawArray(chd_stim,  info)
raw.add_channels( [raw_stimch])

In [ ]:

#force_update_info:  bool
#If True, force the info for objects to be appended to match the values in self. 
# This should generally only be used when adding stim channels for 
# which important metadata won’t be overwritten.

In [ ]:
# define stim channels 
raw.find_events(consecutive = True)

In [ ]:
fname_event = meg_path / 'sample_audvis_raw-eve.fif'
fname_cov = meg_path / 'sample_audvis-cov.fif'

events = mne.read_events(fname_event)
noise_cov = mne.read_cov(fname_cov)

# Standard sample event IDs. These values will correspond to the third column
# in the events matrix.
event_id = {'auditory/left': 1, 'auditory/right': 2, 'visual/left': 3,
            'visual/right': 4, 'smiley': 5, 'button': 32}


# Take only a few events for speed
events = events[:80]

In [ ]:
events

In [ ]:
events.shape  # 80, 3  -- index, ?(duration), code

In [ ]:
# fwd = mne.read_forward_solution(fwd_fname)
# src = fwd['src']


### gen data

In [ ]:
# from collections import OrderedDict as odict
# evts = odict()
import pandas as pd
from config2 import stage2evn2event_ids, env2envcode
# define trial stages
ds = []
d = {'myname':'cursor in home',  'Qname':'rest_phase', 'duration': 464.,
     'evt_ids':[10]}; 
ds+=[d]
d = {'myname':'tgt onset',       'Qname':'target_phase', 'duration': 100.,
     'evt_ids':event_ids_tgt}
ds+=[d]
d = {'myname':'movement',       'Qname':None, 'duration': 100., 
     'evt_ids':[100]}
ds+=[d]
d = {'myname':'feedback',       'Qname':'feedback_phase', 'duration': 232., 
     'evt_ids':event_ids_feedback}
ds+=[d]
d = {'myname':'iti',       'Qname':'iti_phase', 'duration': 400., 
     'evt_ids':[40]}
# normally 1500
ds+=[d]
# 'tgt onset',,
# 'mvt',500,
# 'feedback',232,
# 'iti',1500 
df = pd.DataFrame(ds)
df['err_sens_freq'] = 20.
df['err_sens_coef'] = 1.
df.loc[df['myname'] == 'feedback', 'err_sens_coef'] = 0.5
df.loc[df['myname'] == 'iti', 'err_sens_coef'] = 0.2

display(df)

err_sens_mode = 'indep_inc'
subject = 'sub99' 

ntrials = 30
if err_sens_mode == 'indep_inc':
    err_sens_pt = np.linspace(1, 10, ntrials) # per trial
    org_feedback_pt    = np.linspace(1, 20, ntrials) # per trial
    #error_pt    = np.linspace(1, 20, ntrials) # per trial
trial_infos = []
trial_infos += (ntrials // 2) * [ { 'envname':'stable' } ]
trial_infos += (ntrials // 2) * [ { 'envname':'random' } ]
trial_infos = pd.DataFrame(trial_infos)
trial_infos['tgti'] = np.arange(len(trial_infos)) % 4  #np.random.randint(4, size=nt+1)

epoch_bounds = []
offset_bins = 0
dats = []
evts = []
rows_behav = []
for nt,tinfo in trial_infos.iterrows():    
    tgti = tinfo['tgti']
    envname = tinfo['envname']
    envcode = env2envcode[envname]
    tgtc = stage2evn2event_ids['target'][envname][tgti]
    
    for index,row in df.iterrows():
        dur_s = row['duration'] / 1000.
        dur_bins = int(dur_s * info['sfreq'])
        myname = row['myname']
        freq = row['err_sens_freq']
        coef = row['err_sens_coef']
        times_cur = np.arange(0,dur_bins) / info['sfreq']
        
        # this is what we will encode
        dat = coef * np.sin(2. * np.pi * freq * times_cur)
                
        if nt % 2 == 0:  # in half of trials nothing will happen
            cov_coef_cur_trial = err_sens_pt[nt]
        else:
            cov_coef_cur_trial = 1.
            
        dat *= cov_coef_cur_trial                
        
        dat *= -1
        #data[ offset_bins: offset_bins + dur_bins] = dat
        dats += [dat]
        if row['Qname' ] == 'target_phase':
            evt_id = tgtc
        elif row['Qname' ] == 'feedback_phase':
            evt_id = stage2evn2event_ids['feedback'][envname][0]
        else:
            evt_ids = row['evt_ids']
            evt_id = evt_ids[0]
        # here we put deterministic targets, always first
        evts += [ (offset_bins,0,evt_id)]
        offset_bins += dur_bins
        
    err = 0.33       
    pert = 30
    tgt_ang = 90 + tgti * 30
    fb = org_feedback_pt[nt] + pert
    row = {'trials':nt, 'feedback': fb, 
           'org_feedback':org_feedback_pt[nt], 'perturbation': pert,
                    'error': fb - tgt_ang, 'environment':envcode,
                    'target_inds':tgti, 'target_codes':tgtc, 'target_locs':tgt_ang }
    row['err_sens'] = err_sens_pt[nt]
    #belief
    rows_behav += [ row ]
    
evts = np.array(evts)
print( set(evts[:,2]) )
#vals = 
data = np.hstack(dats)
data_pregen = data

behav_df = pd.DataFrame(rows_behav)

print( 'data.shape = ', data.shape )

print( "info['sfreq'] = ", info['sfreq'] )

In [ ]:
from behav_proc import addBehavCols
behav_df['subject'] = subject
behav_df['target_locs'] = 90.
behav_df['belief'] = behav_df['feedback']
#addBehavCols(behav_df)

behav_df['non_hit_not_adj'] = True
behav_df.to_pickle(pjoin(p,'df.pkl.zip'),compression='zip')

In [ ]:
behav_df

In [ ]:
addBehavCols(df_all)

In [ ]:
data

In [ ]:
def data_fun_tau(times):
    """Generate time-staggered sinusoids at harmonics of 10Hz"""
    global data_pregen
#     n_samp = len(times)
#     window = np.zeros(n_samp)
#     start, stop = [int(ii * float(n_samp) / (2 * n_dipoles))
#                    for ii in (2 * n, 2 * n + 1)]
#     window[start:stop] = 1.
#     n += 1
#     data = 25e-9 * np.sin(2. * np.pi * 10. * n * times)
#     data *= window
#     return data
    return data_pregen

#     parametrized by latency and duration.
#     f = 15  # oscillating frequency, beta band [Hz]
#     sigma = 0.375 * duration
#     sinusoid = np.sin(2 * np.pi * f * (times - latency))
#     gf = np.exp(- (times - latency - (sigma / 4.) * rng.rand(1)) ** 2 /
#                 (2 * (sigma ** 2)))
#     return 1e-9 * sinusoid * gf


#times = raw.times[:int(info['sfreq'] * epoch_duration)]

n_dipoles = 4
add_ECG_EOG = 0

rng = np.random.RandomState(0)
times = np.arange(len(data)  ) / info['sfreq']

stc = simulate_sparse_stc(src, n_dipoles=n_dipoles, times=times,
                          data_fun=data_fun_tau, random_state=rng)
# look at our source data
fig, ax = plt.subplots(1)
ax.plot(times, 1e9 * stc.data.T)
ax.set(ylabel='Amplitude (nAm)', xlabel='Time (sec)')
mne.viz.utils.plt_show()

raw_sim = simulate_raw(info, [stc] * 10, forward=fwd, verbose=True)
#noise_cov = make_ad_hoc_cov(raw_sim.info)
fname_cov = meg_path / 'sample_audvis-cov.fif'
noise_cov = mne.read_cov(fname_cov)

#cov.data *= 10
add_noise(raw_sim, noise_cov, iir_filter=[0.2, -0.2, 0.04], random_state=rng)
if add_ECG_EOG:
    add_ecg(raw_sim, random_state=rng)
    add_eog(raw_sim, random_state=rng)
#raw_sim.plot()

import gc; gc.collect()

raw_sim.add_events(np.array( evts ))

# ids = []
# for ids_cur in list( df['evt_ids'] ):
#     ids += ids_cur
# ids


ids = list(set( evts[:,2] ))
epochs = mne.Epochs(raw_sim, events=evts, event_id = ids)

In [ ]:
raw_sim.add_channels?

In [ ]:
#'/home/demitau/ju_lyon/memerr/data2/sub01_WGPOZPEE/VisuoMotor_no_filter_with_ICA_eve.txt'

In [ ]:
fn_events_full = pjoin(p, 'VisuoMotor_no_filter_with_ICA_eve.txt')
mne.write_events(fn_events_full, evts, overwrite=True)

In [ ]:
p = pjoin(  path_data , 'sub99' );p
raw_sim.save( pjoin(p, 'raw_VisuoMotor_no_filter_with_ICA_broad.fif'), overwrite=1)

# Try to classif 

In [ ]:
#time_locked = 'target'

time_locked = 'feedback'
env = 'stable'
safety_time_bound = 0.05

tmin_cur,tmax_cur = -0.5,0.5

err_sens_non_hit = err_sens

stage2evn2event_ids_str[time_locked][env]

from csp_my import SPoC
from sklearn.linear_model import RidgeCV

n_jobs_SPoC = -1

epochs.load_data()

mne_fit_log_level = 'WARNING'
regression_type = 'Ridge'
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
nb_fold = 7

from scipy.stats import spearmanr

In [ ]:
DEBUG = True
times = epochs.times
# idk why but its really important to have str values here instead of ints
ep = epochs[ stage2evn2event_ids_str[time_locked][env] ]
tmin_safe = tmin_cur + safety_time_bound
tmax_safe = tmax_cur - safety_time_bound

X = ep.pick_types(meg=True, ref_meg=False)._data
if not DEBUG:
    assert X.shape[0] == len(non_hit), (X.shape, len(non_hit) )
wh = (times > tmin_safe) & (times < tmax_safe)

if not DEBUG:
    X = X[non_hit]
X = X[:, :, wh]

##################################  prepare classif

spoc = SPoC(n_components=5, log=True, reg='oas',
            rank='full', n_jobs=n_jobs_SPoC,
            fit_log_level=mne_fit_log_level)
# Regression for classic decoding
if regression_type == 'Ridge':
    alphas = np.logspace(-5, 5, 12)
    est = make_pipeline(spoc, RidgeCV())
elif regression_type == 'xgboost':
    xgb = XGBRegressor(**add_clf_creopts)
    est = make_pipeline(spoc, xgb)
    # Regressions for the B2B

    #param_grid = {
    #    'pca__n_components': [5, 10, 15, 20, 25, 30],
    #    'model__max_depth': [2, 3, 5, 7, 10],
    #    'model__n_estimators': [10, 100, 500],
    #}
    #grid = GridSearchCV(pipeline, param_grid,
    #  cv=5, n_jobs=-1, scoring='roc_auc')
else:
    raise ValueError('wrong regression value')

# Cross-validation
cv = KFold(nb_fold, shuffle=True)

##################################  run classif

y = err_sens_non_hit  # already non_hit
y_preds = np.zeros(len(y))
scores = list()
print(f'Starting CV for regression_type={regression_type}')
nsplit = 0
for train, test in cv.split(X, y):
    print(f'Starting split N={nsplit}')
    with mne.use_log_level(mne_fit_log_level):
        est.fit(X[train], y[train])
    y_preds[test] = est.predict(X[test])
    score = spearmanr(y_preds[test], y[test])
    scores.append(score[0])
    print(scores[0])
    nsplit += 1
diff = np.abs(y - y_preds)

print(np.mean(scores), scores)

In [ ]:
test

In [ ]:
scores, diff

In [ ]:
plt.plot(diff / np.std(y))

In [ ]:
ntrials

In [ ]:
help(add_noise)

# Example

In [ ]:
n_dipoles = 4  # number of dipoles to create
epoch_duration = 2.  # duration of each epoch/event
n = 0  # harmonic number
rng = np.random.RandomState(0)  # random state (make reproducible)


def data_fun(times):
    """Generate time-staggered sinusoids at harmonics of 10Hz"""
    global n
    n_samp = len(times)
    window = np.zeros(n_samp)
    start, stop = [int(ii * float(n_samp) / (2 * n_dipoles))
                   for ii in (2 * n, 2 * n + 1)]
    window[start:stop] = 1.
    n += 1
    data = 25e-9 * np.sin(2. * np.pi * 10. * n * times)
    data *= window
    return data

#     parametrized by latency and duration.
#     f = 15  # oscillating frequency, beta band [Hz]
#     sigma = 0.375 * duration
#     sinusoid = np.sin(2 * np.pi * f * (times - latency))
#     gf = np.exp(- (times - latency - (sigma / 4.) * rng.rand(1)) ** 2 /
#                 (2 * (sigma ** 2)))
#     return 1e-9 * sinusoid * gf


times = raw.times[:int(info['sfreq'] * epoch_duration)]

In [ ]:
stc = simulate_sparse_stc(src, n_dipoles=n_dipoles, times=times,
                          data_fun=data_fun, random_state=rng)
# look at our source data
fig, ax = plt.subplots(1)
ax.plot(times, 1e9 * stc.data.T)
ax.set(ylabel='Amplitude (nAm)', xlabel='Time (sec)')
mne.viz.utils.plt_show()

In [ ]:
raw_sim = simulate_raw(info, [stc] * 10, forward=fwd, verbose=True)
cov = make_ad_hoc_cov(raw_sim.info)
add_noise(raw_sim, cov, iir_filter=[0.2, -0.2, 0.04], random_state=rng)
add_ecg(raw_sim, random_state=rng)
add_eog(raw_sim, random_state=rng)
raw_sim.plot()